# Forecasting 

![extrapolating](https://imgs.xkcd.com/comics/extrapolating.png)

In this lesson, we will practice forecasting using the following methods:  

- Last observed value  
- Simple average  
- Moving average  
- Holt's Linear Trend  
- Previous cycle  


______________________________


We will walk through steps from previous lessons to get the data ready to model

- Acquire data: prepare.acquire_store_data()  
- Prepare data: prepare.prep_store_data()  
- Split data: prepare.split_store_data()  

Then we will forecast and evaluate using each method. 

## Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import env

from datetime import datetime
from sklearn.metrics import mean_squared_error
from math import sqrt
import matplotlib.pyplot as plt
#%matplotlib inline
import seaborn as sns

from pandas.plotting import register_matplotlib_converters

import statsmodels.api as sm
from statsmodels.tsa.api import Holt

## Acquire

We will acquire the store-item-demand data for this lesson from the sql database. 

In [2]:
# define get_connection
df = env.get_tsa()

In [3]:
# assign query to variable 

In [4]:
# read sql query using pd.read_sql()

In [5]:
# take a peek
df.head()

,store_id,item_id,item_upc14,item_upc12,item_brand,item_name,item_price,sale_id,sale_date,sale_amount,store_address,store_zipcode,store_city,store_state
0,1,1,35200264013,35200264013,Riceland,Riceland American Jazmine Rice,0.84,1,2013-01-01,13,12125 Alamo Ranch Pkwy,78253,San Antonio,TX
1,1,1,35200264013,35200264013,Riceland,Riceland American Jazmine Rice,0.84,2,2013-01-02,11,12125 Alamo Ranch Pkwy,78253,San Antonio,TX
2,1,1,35200264013,35200264013,Riceland,Riceland American Jazmine Rice,0.84,3,2013-01-03,14,12125 Alamo Ranch Pkwy,78253,San Antonio,TX
3,1,1,35200264013,35200264013,Riceland,Riceland American Jazmine Rice,0.84,4,2013-01-04,13,12125 Alamo Ranch Pkwy,78253,San Antonio,TX
4,1,1,35200264013,35200264013,Riceland,Riceland American Jazmine Rice,0.84,5,2013-01-05,10,12125 Alamo Ranch Pkwy,78253,San Antonio,TX


## Prepare


1. sale_date to datetime
2. sort values by date
3. set index
4. new field: dollars_sold = sale_amount * item_price
5. rename sale_amount to items_sold to make the two columns easier to understand what the data represents. 
6. resample daily (The original granularity is daily, but there are multiple records of the same days across multiple stores.)
7. remove leap days!

In [6]:
# sale_date to datetime
df = df.assign(ds= pd.to_datetime(df.sale_date))

In [14]:
df.head()

,dollars_sold,item_sold
ds,,
2013-01-01,73844.01,13696
2013-01-02,73570.58,13678
2013-01-03,78169.48,14488
2013-01-04,84467.73,15677
2013-01-05,87621.85,16237


In [7]:
# sort values by date
df = df.sort_values('ds')

In [8]:
# create dollars_sold = sale_amount * item_price
df = df.assign(dollars_sold = df.sale_amount * df.item_price)

In [13]:
df.head()

,dollars_sold,item_sold
ds,,
2013-01-01,73844.01,13696
2013-01-02,73570.58,13678
2013-01-03,78169.48,14488
2013-01-04,84467.73,15677
2013-01-05,87621.85,16237


In [9]:
# create items_sold from sale_amount (rename)
df =df.assign(item_sold = df.sale_amount)

In [10]:
# resample daily, assumming dollars_sold and items_sold
df = df.groupby(['ds'])[['dollars_sold', 'item_sold']].sum()

In [11]:
# set index
#df.set_index('ds')

In [12]:
def prep_data(df):
    return df.assign(ds= pd.to_datetime(df.sale_date)).sort_values('ds').\
    assign(dollars_sold = df.sale_amount * df.item_price).\ # gets you total sales
    assign(item_sold = df.sale_amount).\  # renames the column
    df.groupby(['ds'])[['dollars_sold', 'item_sold']].sum() # this is like a resample by day
         

SyntaxError: unexpected character after line continuation character (<ipython-input-12-3c6d437ee409>, line 3)

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
#df = prep_data(df)

In [ ]:
# remove leap days by takin that day out of the index
df = df[df.index != '2016-02-29']

We will resample to daily, but essentially what we are doing is grouping by the day and aggregating using sum. The original granularity is daily, but there are multiple records of the same days across multiple stores. 

## Split

1. We will use the training proportion method to split.    
2. Identify the total length of the dataframe and multiple by `train_prop` to get the number of rows that equates to the first x% of the dataframe, which equates to the first x% of the time covered in the data.   (`x = train_prop * 100`)  
3. Select row indices from 0 up to the index representing x-percentile for train, and from the index representing x-percentile through the end of the dataframe for test. In both of these, we will reset the index in order to return dataframes sorted by datetime.  
4. Return train and test dataframes.  

In [ ]:
df_size = len(df)
df_size

In [ ]:
# compute num of rows that are 50% of total rows and assign to variable train_size
train_size = int(len(df)* .5)
train_size

In [ ]:
# compute num of rows that are 30% of total rows and assign to variable validate_size
validate_size = int(len(df) * .3)
validate_size

In [ ]:
# make test_size the number of rows remaining (test_size = total # of rows - train_size - validate_size)
test_size = int(len(df)- train_size - validate_size)
test_size

In [ ]:
# compute the row number at which the switch from validate to test happens. 
validate_end_index = train_size + validate_size
validate_end_index

In [ ]:
# split into train, validation, test
train = df[: train_size] # all rows until 912
validate = df[train_size : validate_end_index] 912 to 1459
test = df[validate_end_index :]# 1459 to (1825-1459)

In [ ]:
# sum of train, validate and test = total number of rows? 
len(train) + len(validate) + len(test) == len(df)

Does the first row of original df equate to the first row of train? 

In [ ]:
# test the row starts
print(df.head(1)== train.head(1))

Is the last row of train the day before the first row of validate? And the same for validate to test? 

In [ ]:
# test the split between validate and test
pd.concat([train.tail(1), validate.head(1)]) # the results should be in order from one day to next

In [ ]:
pd.concat([validate.tail(1), test.head(1)]) # the results should be in order from one day to next

Is the last row of test the same as the last row of our original dataframe? 

In [ ]:
# compare the last row of test to last row of df
pd.concat([test.tail(1), df.tail(1)])

Let's plot our data first, viewing where the data is split into train, validate, and test. 

In [ ]:
def plot_samples(target_var):
    ''' this function will plot the train, validate and test values for a single variable 
    '''
    plt.figure(figsize=(12,4))
    plt.plot(train[target_var])
    plt.plot(validate[target_var])
    plt.plot(test[target_var])
    plt.title(target_var)
    plt.show()

In [ ]:
# plot the data points, color by train, validate, test
#col = 'dollars_sold'
for col in train.columns:
    plot_samples(col)

plt.show()

Before we try out different methods for forecasting sales and number of items sold, let's create a couple of functions that will be helpful in evaluating each of the methods that follow. 

`evaluate()` will compute the Mean Squared Error and the Rood Mean Squared Error to evaluate.  

In [ ]:
# define evaluation function to compute rmse

def evaluate(target_var):
    '''
    the evaluate function will take in the actual values in the validate and the predicted values
    '''
    rmse = round(sqrt(mean_squared_error(validate[target_var], yhat_df[target_var])), 0)
    return rmse

`plot_and_eval()` will use the evaluate function and also plot train and test values with the predicted values in order to compare performance. 

In [ ]:
# plot and evaluate: plot
def plot_and_eval(target_var):
    '''
    a function to evaluate forecasts by computing the rmse and plot train and validate along with predictions
    '''
    plot_samples(target_var)
    plt.plot(yhat_df[target_var])
    plt.title(target_var)
    rmse = evaluate(target_var)
    print(target_var, '--RMSE:  {: 0f}'.format(rmse))
    plt.show()

Write `append_eval_df(model_type)` to append evaluation metrics for each model type, target variable, and metric type, along with the metric value into our `eval_df` data frame object. Which we will create an empty `eval_df` dataframe object to start. 

In [ ]:
# Create the empty dataframe with model_type, target_var, rmse
eval_df = pd.DataFrame(columns = ['model_type', 'target_var','rmse'])

In [ ]:
# check out the data frame
eval_df

In [ ]:
# Define function to store rmse for comparison purposes
def append_eval_df(model_type, target_var):
    '''
    this function is going to take in the model_type as a string, the target variable as a string,
    and run the evaluate() function to compute the rmse,
    and append to the dataframe a row with the model_type, target_var, and rmse. 
    It will return the new dataframe.
    '''
    rmse = evaluate(target_var)
    d = {'model_type': [model_type], 'target_var': [target_var], 'rmse': [rmse]}
    d = pd.DataFrame(d)
    return eval_df.append(d, ignore_index = True)

## Forecast 

### Last observed value

The simplest method for forecasting is to predict all future values to be the last observed value.  

**Make Predictions**

In [ ]:
# create var 'items' with last observed value
items = train['item_sold'][-1]
items

In [ ]:
# create dollars 'items' with last observed value
dollars = round(train['dollars_sold'][-1],2)
dollars

In [ ]:
# make predictions by adding those values to new dataframe yhat_df
yhat_df = pd.DataFrame({'item_sold': [items], 'dollars_sold': [dollars]}, index = validate.index)
yhat_df.head(2)

You can see, when peeking into yhat_df, that every predicted value is the same.  

**Plot Actual vs. Predicted Values**

Now, let's plot actual and predicted values

In [ ]:
for col in train.columns:
    plot_and_eval(col)

**Evaluate** 

Evaluate using MSE and RMSE, and add evaluation metrics to `eval_df`

In [ ]:
for col in train.columns:
    eval_df = append_eval_df(model_type = 'last_observed_value', target_var = col)

### Simple Average

Take the simple average of historical values and use that value to predict future values.   

This is a good option for an initial baseline. Every future datapoint (those in 'test') will be assigned the same value, and that value will be the overall mean of the values in train. 

**Make Predictions**

In [ ]:
items = round(train['item_sold'].mean(), 2)
dollars = round(train['dollars_sold'].mean(), 2)

In [ ]:
def make_predictions(items, dollars):
    yhat_df = pd.DataFrame({'item_sold': [items], 'dollars_sold':[dollars]}, index = validate.index)
    return yhat_df

**Plot Actual vs. Predicted Values**

Now, let's plot and evaluate the performance of our time series model using **Simple Average**

In [ ]:
yhat_df = make_predictions(items, dollars)
yhat_df

**Evaluate**

Evaluate using MSE and RMSE, and add evaluation metrics to `eval_df`

In [ ]:
for col in train.columns:
    plot_and_eval(col)

### Moving Average

In this example, we will use a 30-day moving average to forecast. In other words, the average over the last 30-days will be used as the forecasted value. 

**Make Predictions**

In [ ]:
yhat_df = make_predictions(items, dollars)

In [ ]:
yhat_df.head(3)

**Plot Actual vs. Predicted Values**

Now, let's plot and evaluate the performance of our time series model using **Moving Average**

In [ ]:
for col in train.columns:
    plot_and_eval(col)

In [ ]:
period = 30
items = round(train['item_sold'].rolling(period).mean().iloc[-1])
dollars = round(train['dollars_sold'].rolling(period).mean().iloc[-1])
items

**Evaluate**

Evaluate using MSE and RMSE, and add evaluation metrics to `eval_df`

In [ ]:
for col in train.columns:
    eval_df = append_eval_df(model_type="30d moving average", target_var = col)

In [ ]:
# append the evaluation results to our eval_df for each target variable so we can 
for col in train.columns:
    eval_df = append_eval_df(model_type = '30d moving average', target_var = col)

In [ ]:
#eval_df[eval_df.target_var == 'dollars sold']
#eval_df[eval_df.target_vaar == 'item_sold']

#find the min value
eval_df_sort_values(by=['rmse'])

Let's try out several other values for periods:

In [ ]:
period = 200
items = round(train['item_sold'].rolling(period).mean().iloc[-1])
dollars = round(train['dollars_sold'].rolling(period).mean().iloc[-1])
items

In [ ]:
period = 15
items = round(train['item_sold'].rolling(period).mean().iloc[-1])
dollars = round(train['dollars_sold'].rolling(period).mean().iloc[-1], 2)
yhat_df = make_predictions(items, dollars)
for col in train.columns:
    eval_df = append_eval_df(model_type="15d moving average", target_var = col)
eval_df

Which is best so far? 

In [ ]:
# get the min rmse for each variable
eval_df.sort_values(by=['rmse']).groupby('target_var').first()

In [ ]:
# filter only the rows that match those rmse to find out 
# which models are best thus far


### Holt's Linear Trend

Exponential smoothing applied to both the average and the trend (slope).  

- $\alpha$ / smoothing_level: smoothing parameter for mean. Values closer to 1 will have less of a smoothing effect and will give greater weight to recent values.   
- $\beta$ / smoothing_slope: smoothing parameter for the slope. Values closer to 1 will give greater weight to recent slope/values. 


**Seasonal Decomposition**

First, let's take a look at the seasonal decomposition for each target. 

In [ ]:
import statsmodels.api as sm

for col in train.columns:
    print(col, '\n')
    _ = sm.tsa.seasonal_decompose(train[col].resample('W').mean()).plot()
    plt.show()

#### Basic Holt's Linear Trend

**Make Predictions**

Now, like we would when using sklearn, we will create the Holt object, fit the model, and make predictions. 

Holt: 

- exponential = True/False (exponential vs. linear growth, additive vs. multiplicative)

fit: 

- smoothing_level ($\alpha$): value between (0,1)
- smoothing_slope ($\beta$): value between (0,1)

In [ ]:
for col in train.columns:
    # Making the model
    model = Holt(train[col], exponential = False)
    
    # Fitting the model
    model = model.fit(smoothing_level = .5, smoothing_slope = .5, optimized = False)
    
    # Making predictions 
    yhat = model.predict(start = validate.index[0], end = validate.index[-1])
    
    yhat_df[col] = round(yhat, 0)

In [ ]:
# for col in train.columns:
#     model = Holt(train['item_sold'], exponential = False)

#     model = model.fit(smoothing_level = .5, 
#                  smoothing_slope = .5,
#                  optimized = False)
#     # predict/forecast providing the start and end dates
#     yhat_items = model.predict(start = validate.index[0], end = validate.index[-1])
    
#     yhat_df[col] = round(yhat, 2)

**Plot Actual vs. Predicted Values**

In [ ]:
for col in train.columns:
        eval_df = append_eval_df(model_type = 'Holts', target_var = col)
        plot_and_eval(target_var = col)

In [ ]:
for col in train.columns:
    # Making the model
    model = Holt(train[col], exponential = False)
    
    # Fitting the model
    model = model.fit(smoothing_level = .1, smoothing_slope = .1, optimized = False)
    
    # Making predictions 
    yhat = model.predict(start = validate.index[0], end = validate.index[-1])
    
    yhat_df[col] = round(yhat, 0)
    
for col in train.columns:
        eval_df = append_eval_df(model_type = 'Holts L=.1 S=.1', target_var = col)
        plot_and_eval(target_var = col)

In [ ]:
yhat_df.head()

**Evaluate**

In [ ]:
# eval_df = eval_df[eval_df.model_type != 'Holts']

In [ ]:
eval_df

### Predict Based on Previous Cycle

Take all the 2016 data points, compute the daily delta, year-over-year, average that delta over all the days, and adding that average to the previous year's value on a day will give you the forecast for that day. 

If a primary cycle is weekly, then you may want to do this on a week-over-week cadence. 

In the below example:  
1. Compute the 365 average year over year differences from 2013 through 2015
2. Add that average delta to the values during 2015. 
3. Set the index in your yhat dataframe to represent the dates those predictions are make for. 

Let's get started....

**Re-split data**

In [ ]:
train = df[:'2015']
validate = df['2016']
test = df['2017']

In [ ]:
# year difference
train.diff(365).mean()

**Make Predictions**

In [ ]:
# take the values for each day in 2015 and add the average year over year(yoy)(y/y)
yhat_df = train['2015'] + train.diff(365).mean()

In [ ]:
# set yhat_df to index of validate


In [ ]:
yhat_df = yhat_df.set_index(validate.index)


In [ ]:
eval_df.sort_values('rmse').groupby('target_var').first()

In [ ]:
# set yhat_df to index of validate
yhat_df = yhat_df.set_index(validate.index)


In [ ]:

# set yhat_df to index of validate
yhat_df.head()

**Plot and Evaluate**

In [ ]:
for col in train.columns:
    eval_df = append_eval_df(model_type = 'yoy-diff', target_var = col)
    plot_and_eval(target_var = col)

In [ ]:
compare_df = pd.concat([yhat_df.item_sold, validate.item_sold], axis = 1)
compare_df.columns = ['yhat_items', 'actual_items']
compare_df['error'] = compare_df.actual_items - compare_df.yhat_items
compare_df['squared_error'] = compare_df.error * compare_df.error
#compare_df[compare_df.squared_error.mean()]
compare_df

## Conclusion

Which model did the best? 

In [ ]:
# get the min rmse for each variable


# filter only the rows that match those rmse to find out 
# which models are best thus far


Let's test it out on our out-of-sample data

We will be using train + validate to predict test. 

In [ ]:
# must use same parameters we used from train
yhat_ = validate + train.diff(365).mean()

# set index to that of test
yhat_df.index = test.index


In [ ]:
for col in train.columns:
    plot_and_eval(col)
    append_eval_df(model_type = 'yoy-diff-test', target_var = col)

## Exercises

The end result of this exercise should be a Jupyter notebook named `model`.

Using [saas.csv](https://ds.codeup.com/saas.csv) or log data from API usage or store_item_sales

1. Split data (train/validate/test) and resample by any period, except daily, and aggregate using the sum. 
2. Forecast, plot and evaluate using each of the 4 parametric based methods we discussed:
    - Simple Average
    - Moving Average
    - Holt's Linear Trend Model
    - Based on previous year/month/etc., this is up to you.

Optional: Using store item demand

1. Predict 2018 total **monthly** sales for a single store and/or item by creating a model.
2. Return a dataframe with the month, store_id, y-hat, and the confidence intervals (y-hat lower, y-hat upper).
3. Plot the 2018 monthly sales predictions.